In [ ]:
#fork from https://github.com/AYLIEN/gan-intro/blob/master/gan.py#L49

In [2]:
import numpy as np
import tensorflow as tf

In [5]:
class GeneratorDistribution(object):
    def __init__(self , range_):
        self.range_ = range_
    
    def sample(self , N):
        return np.linspace(-self.range_ , self.range_ , N) + np.random.random(N)*0.01

In [8]:
def linear(input , output_dim , scope=None , stddev=1.0):
    with tf.variable_scope(scope or 'linear'):
        w = tf.get_variable(
        'w',
        [input.get_shape()[1] , output_dim],
        initializer = tf.random_normal_initializer(stddev=stddev))
    
        b = tf.get_variable(
        'b',
        [output_dim],
        initializer = tf.constant_initializer(0.0))
        
        return tf.matmul(input , w) + b

In [9]:
def generator(input ,h_dim):
    h0 = tf.nn.softplus(linear(input , h_dim , 'g0'))
    h1 = linear(h0 , 1 , 'g1')
    
    return h1

In [10]:
def minibatch(input , num_kernels = 5 , kernel_dim = 3):
    x = linear(input , num_kernels*kernel_dim , scope='minibatch' , stddev=0.02)
    activation = tf.reshape(x , (-1 , num_kernels , kernel_dim))
    
    diffs = tf.expand_dims(activation , 3)-tf.expand_dims(tf.transpose(activation , [1,2,0]) , 0)
    
    abs_diffs = tf.reduce_sum(tf.abs(diffs) , 2)
    
    minibatch_features = tf.reduce_sum(tf.exp(-abs_diffs) , 2)
    
    return tf.concat([input , minibatch_features] , 1)

In [11]:
def discriminator(input , h_dim , minibatch_layer = True):
    h0 = tf.nn.relu(linear(input , h_dim*2 , 'd0'))
    h1 = tf.nn.relu(linear(h0 , h_dim*2 , 'd1'))
    
    if minibatch_layer:
        h2 = minibatch(h1)
    else:
        h2 = tf.nn.relu(linear(h1 , h_dim*2 , scope='d2'))
    
    h3 = tf.sigmoid(linear(h2 , 1 , scope='d3'))
    
    return h3

In [12]:
def optimizer(loss , var_list):
    learning_rate = 0.001
    step = tf.Variable(0 , trainable=False)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss , global_step=step,var_list=var_list)
    
    return optimizer

In [13]:
def log(x):
    return tf.log(tf.maximum(x , 1e-5))

In [14]:
class GAN(object):
    def __init__(self , params):
        with tf.variable_scope('G'):
            self.z = tf.placeholder(tf.float32 , shape=(params.batch_size , 1))
            self.G = generator(self.z , params.hidden_size)
            
        self.x = tf.placeholder(tf.float32 , shape = (params.batch_size , 1))
        with tf.variable_scope('D'):
            self.D1 = discriminator(self.x , params.hidden_size , params.minibatch)
        
        
        self.loss_d = tf.reduce_mean(-log(self.D1)-log(1-self.D2))
        self.loss_g = tf.reduce_mean(-log(self.D2))
        
        vars = tf.trainable_variables()
        self.d_params = [v for v in vars if v.name.startswith('D/')]
        self.g_params = [v for v in vars if v.name.startswith('G/')]
        
        self.opt_d = optimizer(self.loss_d , self.d_params)
        self.opt_g = optimizer(self.loss_g , self.g_params)